In [ ]:
#Author : Dalyah
#Modifications: 11-4-2018 till 13-4-2018 

#Pkg.update("ImageView")
using ImageView
using Plots
using Knet;
using Compat,GZip 


## Download and read data

In [ ]:
# Generate Random Data and Minibatching
# Input img:128*128
#Batch Size= 128

xtrn= 0.1*rand(128,128,1,5000);
ytrn= 0.1*rand(48, 5000);


xtst= 0.1*rand(128,128,1,1000);
ytst= 0.1*rand(48, 1000);

Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
#Atype= Array{Float32}

dtrn = minibatch(xtrn,ytrn,120; xtype=Atype, ytype=Atype); 
dtst = minibatch(xtst,ytst,120; xtype=Atype, ytype=Atype); 



In [ ]:
#Extracting Zip Files
using InfoZIP
using ZipFile

#InfoZIP.unzip("Training.zip", "Data/")
#InfoZIP.unzip("Testing.zip", "Data/")

#They are already extracted on my AWS image: HandPosE_Dalyah_v6

## Sample Image Data Extraction

In [ ]:
## imgDraft file

using Images
using Colors
images = Any[]
rootpath="imgDraft" # should change based on path on Machine you work on
arr1 = ["1","2"]
arr2 = ["123","321"]
for i =1:length(arr1)
    for j=1:length(arr2)
        path2 = string(rootpath,"/",arr1[i],"/",arr2[j])
        x =readdir(path2) #save all img names in arr1[i]/arr2[j] in x var
        for k=1:length(x)
            path3 = string(path2,"/",x[k])
            
            img = load(path3)
            #Transfer to gray scale
            img2=Float64.(Gray.(img))
            #imshow(img)
            #imshow(img2)
            println(typeof(img))
            println(size(img2))
            
            #Resize img to (128,128)
            sz=(128,128)
            σ = map((o,n)->0.75*o/n, size(img), sz)
            kern = KernelFactors.gaussian(σ)   # from ImageFiltering
            img3 = imresize(imfilter(img, kern, NA()), sz)
            println(size(img3))
            #imshow(img3)
            #Convert to array
            #img3 = convert(Array{float64},img2) # Check Visualization Lab
            push!(images,img2)
        end
    end
end
println("Final")
println(typeof(images))
for x in images
    println(size(x))
    println(typeof(x))
end
println(length(images))


In [ ]:
#Reading Training and Testing img labels

#HOW THE DATA IS ORGANIZED

#Each line is corresponding to one image.
#Each line has 16x3 numbers, which indicates (x, y, z) of 16 joint locations. Note that these are joint CENTRE locations.
#Note that (x, y) are in pixels and z is in mm.
#The order of 16 joints is Palm, Thumb root, Thumb mid, Thumb tip,
#Index root, Index mid, Index tip, Middle root, Middle mid, Middle tip, Ring root,
#Ring mid, Ring tip, Pinky root, Pinky mid, Pinky tip.

function read_labels(label_path)
    labels=Any[]
    open(label_path) do f
         # loop through lines in file (images)
            #loop through all words in a line (joint locations)
                #skip first word
                #stack all joint locations on top of each other
                #return imgjoints vector(48,1) or (49, 1) as first element is the name of img
            #horizantally concatanate imgjoints into labels matrix (48, #lines)
        
        for ln in eachline(f)
            img_joints= Any[]
            word= split(ln)
            for i in 1:49
                push!(img_joints, word[i])
            end
            #println(summary(img_joints))

            if length(labels) ==0
                labels=img_joints
                #println(labels)
            else
                labels= hcat(labels, img_joints)
            end
        end
        println("labels Dimentions")
        println(summary(labels))
        println(length(labels))
        println(labels[1,2])
    end
    return labels
end


In [ ]:
trn_labelpath= "Data/Training/labels.txt"
tst1_labelpath= "Data/Testing/test_seq_1.txt"
tst2_labelpath= "Data/Testing/test_seq_2.txt"

trn_labels= read_labels(trn_labelpath) # (49, )
tst1_labels= read_labels(tst1_labelpath) # (49, )
tst2_labels= read_labels(tst2_labelpath) # (49, )

#The first label should be reomved to keep labels size (48, )

println(map(summary,(trn_labels, tst1_labels, tst2_labels)))



In [ ]:
#8 Samples for the training , 4 is minibatch size to make sure the model is working fine 
#next assignment I will Train the model on the whole dataset

xtrn= images
ytrn= 0.1*rand(48, 8);


Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32}
#Atype= Array{Float32}

dtrn = minibatch(xtrn,ytrn,4; xtype=Atype, ytype=Atype); 
#dtst = minibatch(xtst,ytst,120; xtype=Atype, ytype=Atype); 


## Part of the REAL Data Extraction

In [ ]:
## Extract Data

In [ ]:
#Checking Diemtionalities

#Desired Dim for (xtrn, ytrn, xtst, ytst)
#("128×128×1×60000 Array{Float32,4}", "60000-element Array{UInt8,1}",
#"128×128×1×10000 Array{Float32,4}", "10000-element Array{UInt8,1}")

println(map(summary,(xtrn,ytrn,xtst,ytst)))
println("Num of MiniBatches")
println(length(dtrn))# Return number of MiniBatches size

println("xtst")
for x in xtst
    println(size(x))
    println(summary(x))
    println(typeof(x))
    break;
    #println(size(y)) 
end
println("dtst")
for (x, y) in dtst
    println(size(x)) 
    println(size(y)) 
end

println("xtrn")
for x in xtrn
    println(size(x)) 
    #println(size(y)) 
end
println("dtrn")
for (x, y) in dtrn
    println(size(x)) 
    println(size(y)) 
end

#to return Dimentionality of each Minibatch
println("Dim of first MiniBtch")
println(summary(dtrn))
println(summary(dtrn))
#(x, y) = first(dtrn);
println("hh")
println(dtrn)
println(xtrn)
println(summary(y))

#for (x, y) in dtrn
 #   
#end


## CNN Model

1. Predict Function
2. Loss Function
3. Loss Gradient
4. Prameter Intialization
5. Trian Function


## Hyber Parameters

Batch size: 128

Momentum: 0.9

Weight decay: 0.001

Learning  rate: 0.01

Epoch number: 100

PCA Components: 30

## LOSS

In [ ]:
#Loss Functions

# NLL Loss
#loss(w,x,ygold) = nll(predict(w,x),ygold)

#Mean Loss
#loss(w,x,y)= mean(abs2,y-predict(w,x))
function loss(w,x,y)
    #y= convert(Array{Float32,2},y)
    #y= convert(Knet.KnetArray{Float32,4},y)
    return mean(abs2,y-predict(w,x))
end

#Huber Loss
function huber_loss(w, x, y ; e=0.4)
    ypred=predict(w,x)
    N= size(y,1)
    E= ill(e, N)
    com=abs(y - ypred) .<= E
    
    out = true
    for i in 1:length(s)
        if  s[i] != true
            out=false
        end
    end
    if out
        lo= 0.5* (y- ypred).^2
    else
        lo=e* abs(y - ypred) - 0.5 * (E .^2)
        
    end
    return lo
end

#Gradient Loss
lossgradient = grad(loss);

## CNN Model Predict 

In [ ]:
#Conv 8(5*5) , Max Pool (4*4), Conv 8(5*5), Max Pool (2*2), Conv 8(3*3), 2 FC (1024),droupout, PCA (30), FC(48)
function predict(w,x)
    #win=[4,0,2]
    #x= map(Atype,x)
    #n=length(w)-8
    #for i=1:2:n # we are jumping 2 steps since alwase we have w then b, then w  then b....
     #   x = pool(relu.(conv4(w[i],x) .+ w[i+1]); window=win[i]) 
    #end
    #i=n+1
    #x= relu.(conv4(w[i],x) .+ w[i+1])
    #for i=n+3:2:length(w)-2 # Fully connected layers
    #    x = relu.(w[i]*mat(x) .+ w[i+1])
    #end
    # Add PCA layer later
    #print(size( w[end-1]*mat(x) .+ w[end]  ))
    #return w[end-1]*mat(x) .+ w[end]
    
    #First Conv Layer
    x = pool(relu.(conv4(w[1],x) .+ w[2]); window=4) 
    
    #Second Conv Layer
    x = pool(relu.(conv4(w[3],x) .+ w[4]); window=2) 
    
    #Third Conv Layer
    x = relu.(conv4(w[5],x) .+ w[6])
    
    #FC 1
    x = relu.(w[7]*mat(x) .+ w[8])
    
    #FC 2
    x = relu.(w[9]* x .+ w[10])
    
    #OUTPUT
    return w[11]* x .+ w[12]
    
end

## Parameter Intialization

In [ ]:

#DimensionMismatch("((1024, 103968), (968, 120), (1024, 120))")

wcnn=map(Atype, [ 0.1*randn(5,5,1,8),  zeros(1,1,8,1), 
        0.1*randn(5,5,8,8),  zeros(1,1,8,1),
        0.1*randn(3,3,8,8),  zeros(1,1,8,1), #103968
        0.1*randn(1024,968),  zeros(1024,1),
        0.1*randn(1024,1024),  zeros(1024,1), # add PCA Layer parameters later
        0.1*randn(48,1024), zeros(48,1)])

In [ ]:
#TRAIN CNN Function

function train!(w, data; lr=.01)
    for (x,y) in data
        println("sqr error loss")
        print(loss(w,x,y))
        println("Huber Loss")
        #print(huber_loss(w,x,y))
        dw = lossgradient(w, x, y)
        println(size(dw))
        for i in 1:length(w) # Loop  over W and B
           @time print(w[i] -= lr * dw[i])
        end
    end
    
    return w
end

## TRAINING

In [ ]:
#MAIN
@time cnnmodels = [ copy(train!(wcnn, dtrn)) for epoch=1:5 ];

In [ ]:
#CNN Loss
#@time trncnnloss = [mean(abs2,ytrn-predict(w,xtrn)) for w in cnnmodels ]; 
#@time tstcnnloss = [ mean(abs2,ytst-predict(w,xtst)) for w in cnnmodels ]; 

trncnnloss=0
tstcnnloss =0
c=1
trncnnlosses= []
@time for w in cnnmodels
    w= map(Atype, w)
    for (x,y) in dtrn
        #trncnnloss =+ mean(abs2,y-predict(wcnn,x))
        trncnnloss =+ loss(w,x,y)
        c+=1
    end
    push!(trncnnlosses, trncnnloss) #trncnnloss/c
end
    
    
#Testing Set   
#for (x,y) in dtst
    #tstcnnloss =+ loss(wcnn,x,y)
#end

print(trncnnlosses)
#print(tstcnnloss/c)

In [ ]:
#Plotting Loss

plot( [trncnnlosses],ylim=(.000,.010),labels=[ :trncnnlosses ],xlabel="Epochs",ylabel="Loss") 

In [ ]:
##Accuracy

#trnAcc= accuracy(wcnn,dtrn,predict)
#tstAcc=accuracy(wcnn,dtst,predict)

#trnAcc=0
#tstAcc =0
#c=1

#for (x,y) in dtrn
   # y= convert(Array{Float32,2},y)
   # x= convert(Array{Float32,2},x)
   # trnAcc =+ accuracy(predict(wcnn,x),y)
    #c+=1
#end

#for (x,y) in dtst
#    y= convert(Array{Float32,2},y)
#    tstAcc = + accuracy(predict(wcnn,x),y) 
#end

#print(trnAcc/c)
#print(tstAcc/c)

#print(trnAcc)